In [92]:
# 构建模型
# 导入GBDT算法
from sklearn.ensemble import GradientBoostingRegressor #梯度提升算法
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split,KFold,cross_val_score #划分数据集包，交叉验证包
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn import neighbors
from sklearn.ensemble import StackingRegressor
from sklearn import ensemble
from sklearn.neural_network import MLPRegressor
from sklearn import tree
import xgboost

In [51]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [93]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
# 特征标准化
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))
# # 若无法获得测试数据，则可根据训练数据计算均值和标准差
# numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
# all_features[numeric_features] = all_features[numeric_features].apply(
#     lambda x: (x - x.mean()) / (x.std()))
# # 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
# all_features[numeric_features] = all_features[numeric_features].fillna(0)
# ExterQual ExterCond  BsmtQual BsmtCond BsmtExposure BsmtFinType1 BsmtFinType2 HeatingQC KitchenQual FireplaceQu GarageQual GarageCond PoolQC

In [94]:
qual_map={'Po':1.0,'Fa':2.0,'TA':3.0,'Gd':4.0,'Ex':5.0}#ExterQual  ExterCond HeatingQC KitchenQual
qual2_map={'NA':0.0,'Po':1.0,'Fa':2.0,'TA':3.0,'Gd':4.0,'Ex':5.0}#BsmtQual BsmtCond FireplaceQu GarageQual GarageCond
qual3_map={'NA':0.0,'No':1.0,'Mn':2.0,'Av':3.0,'Gd':4.0}#BsmtExposure
qual4_map={'NA':0.0,'Unf':1.0,'LwQ':2.0,'Rec':3.0,'BLQ':4.0,'ALQ':5.0,'GLQ':6.0}#BsmtFinType1 BsmtFinType2
qual5_map={'NA':0.0,'Fa':1.0,'TA':2.0,'Gd':3.0,'Ex':4.0}#PoolQC
all_features['ExterQual']=all_features['ExterQual'].map(qual_map)
all_features['ExterCond']=all_features['ExterCond'].map(qual_map)
all_features['HeatingQC']=all_features['HeatingQC'].map(qual_map)
all_features['KitchenQual']=all_features['KitchenQual'].map(qual_map)
all_features['BsmtQual']=all_features['BsmtQual'].map(qual2_map)
all_features['BsmtCond']=all_features['BsmtCond'].map(qual2_map)
all_features['FireplaceQu']=all_features['FireplaceQu'].map(qual2_map)
all_features['GarageQual']=all_features['GarageQual'].map(qual2_map)
all_features['GarageCond']=all_features['GarageCond'].map(qual2_map)
all_features['BsmtExposure']=all_features['BsmtExposure'].map(qual3_map)
all_features['BsmtFinType1']=all_features['BsmtFinType1'].map(qual4_map)
all_features['BsmtFinType2']=all_features['BsmtFinType2'].map(qual4_map)
all_features['PoolQC']=all_features['PoolQC'].map(qual5_map)
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [95]:
all_features = pd.get_dummies(all_features, dummy_na=True)
n_train = train_data.shape[0]
train_features = all_features[:n_train].values
test_features = all_features[n_train:].values
train_labels = train_data.SalePrice.values.reshape(-1, 1)

In [4]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
# 特征标准化
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))
# 若无法获得测试数据，则可根据训练数据计算均值和标准差
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 在标准化数据之后，所有均值消失，因此我们可以将缺失值设置为0
all_features[numeric_features] = all_features[numeric_features].fillna(0)
# “Dummy_na=True”将“na”（缺失值）视为有效的特征值，并为其创建指示符特征
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape
n_train = train_data.shape[0]
train_features = all_features[:n_train].values
test_features = all_features[n_train:].values
train_labels = train_data.SalePrice.values.reshape(-1, 1)

In [96]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)
# 定义验证函数,使用5折交叉验证，采用均方根误差判别，返回均方根误差
def rmse_cv(model):
    kf = KFold(n_splits=5, shuffle=True, random_state=0)# 划分成五分
    cvs=cross_val_score(model,train_features,train_labels,scoring="neg_mean_squared_log_error",cv = 5) #这里是输入了x,y
    #这里的cv参数为分配器, 返回每一次交叉验证的估计量分数数组
    #这里返回的是每组方差的负数
    # print(cvs)
    #返回误差
    rmse= np.sqrt(-cvs)
    # print(rmse)
    return rmse

In [56]:
# 构建一个GBDT的实例用来验证其准确率
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.005,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10,
                                   loss='huber', random_state =5)
#loss：损失函数，ls是最小二乘，lad是最小绝对偏差，huber是前两者的结合
#learning_rate:学习率决定了参数移动到最优值的速度快慢，学习率过大可能越过最优值，学习率过小可能会减缓收敛速度。
#max_depth:限制了树的节点，可以调整这个数字来获得最优表现
#max_features:  这个max_features=sqrt(n_features)
#min_samples_split：每个叶子节点所需要的最小样本数
#n_estimators： 提升学习过程被终止时基分类器的最大数量。一个强大的参数，一般都是先调整它
# random_state：这里的random_state就是为了保证程序每次运行都分割一样的训练集和测试集。否则，同样的算法模型在不同的训练集和测试集上的效果不一样。
#当你用sklearn分割完测试集和训练集，确定模型和初始参数以后，你会发现程序每运行一次，都会得到不同的准确率，无法调参。这个时候就是因为没有加random_state。加上以后就可以调参了。

score1 = rmse_cv(GBoost)
# 输出五个均方根误差的平均值RSME和其标准差SD,保留4位小数
print("Gradient Boosting score: RSME={:.4f} (SD={:.4f})\n".format(score1.mean(),score1.std()))#Gradient Boosting score: RSME=0.1247 (SD=0.0123) Gradient Boosting score: RSME=0.1246 (SD=0.0114)

D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\ensemble\_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of 

Gradient Boosting score: RSME=0.1246 (SD=0.0114)



In [82]:
# 构建一个GBDT的实例用来验证其准确率
GBoost = GradientBoostingRegressor(random_state =6)
#loss：损失函数，ls是最小二乘，lad是最小绝对偏差，huber是前两者的结合
#learning_rate:学习率决定了参数移动到最优值的速度快慢，学习率过大可能越过最优值，学习率过小可能会减缓收敛速度。
#max_depth:限制了树的节点，可以调整这个数字来获得最优表现
#max_features:  这个max_features=sqrt(n_features)
#min_samples_split：每个叶子节点所需要的最小样本数
#n_estimators： 提升学习过程被终止时基分类器的最大数量。一个强大的参数，一般都是先调整它
# random_state：这里的random_state就是为了保证程序每次运行都分割一样的训练集和测试集。否则，同样的算法模型在不同的训练集和测试集上的效果不一样。
#当你用sklearn分割完测试集和训练集，确定模型和初始参数以后，你会发现程序每运行一次，都会得到不同的准确率，无法调参。这个时候就是因为没有加random_state。加上以后就可以调参了。

# score1 = rmse_cv(GBoost)
# # 输出五个均方根误差的平均值RSME和其标准差SD,保留4位小数
# print("Gradient Boosting score: RSME={:.4f} (SD={:.4f})\n".format(score1.mean(),score1.std()))#Gradient Boosting score: RSME=0.1315 (SD=0.0097) Gradient Boosting score: RSME=0.1292 (SD=0.0101)


In [98]:
#SVM回归
model_SVR = svm.SVR()
#knn回归
model_KNN = neighbors.KNeighborsRegressor()
model_ML = MLPRegressor(hidden_layer_sizes=(100,50), activation='relu',solver='adam',
    alpha=0.01,max_iter=200)
#Adaboot回归
#决策树回归
xgb_reg = xgboost.XGBRegressor()
model_DT = tree.DecisionTreeRegressor(random_state=50)
model_ABR = ensemble.AdaBoostRegressor(random_state=70)
model_RFR = ensemble.RandomForestRegressor(random_state=30)
estimators = [('ml',model_ML),('xgb',xgb_reg)]
stack = StackingRegressor(estimators=estimators,final_estimator=GBoost)
score1 = rmse_cv(stack)
# 输出五个均方根误差的平均值RSME和其标准差SD,保留4位小数
print("Gradient Boosting score: RSME={:.4f} (SD={:.4f})\n".format(score1.mean(),score1.std()))

D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\ensemble\_stacking.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
D:\application\anaconda3\envs\d2l\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
D

Gradient Boosting score: RSME=0.1343 (SD=0.0098)



In [97]:
xgb_reg = xgboost.XGBRegressor()
score1 = rmse_cv(xgb_reg)
# 输出五个均方根误差的平均值RSME和其标准差SD,保留4位小数
print("Gradient Boosting score: RSME={:.4f} (SD={:.4f})\n".format(score1.mean(),score1.std()))

Gradient Boosting score: RSME=0.1381 (SD=0.0069)

